# SpinEDA: Análisis exploratorio, NLP y + de la obra de Luis alberto Spinetta

Disclaimer: El siguiente análisis tiene como objetivos ayudarme a aprender e incorporar diversos conceptos de python y data science. Es probable que haya errores, formas de hacer mejor las cosas, etc. Se aceptan sugerencias de todo tipo.

# Libraries y funciones

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spot_config import CLIENT_ID, CLIENT_SECRET
from pprint import pprint

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
#Default fig size para plt
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 6
plt.rcParams["figure.figsize"] = fig_size

In [3]:
def print_full(df):
    """
    Mostrar el dataframe completo, no solo el head y tail.
    
    df: Pandas.DataFrame object
    """
    #pd.set_option('display.max_rows', len(df))
    #display(df)
    #pd.reset_option('display.max_rows')
    with pd.option_context("display.max_rows", len(df), "display.max_columns", len(df.columns)):
        display(df)

***

# Spotify "scraping"

La idea es:
- Aprender a usar la API de spotify, en este a travaés de la librería Spotipy (python friendly API)
- Inicialmente pensé en usar la tabla de [Wikipedia](https://es.wikipedia.org/wiki/Anexo:Canciones_de_Luis_Alberto_Spinetta#1966_-_2020) que lista sus canciones, pero encontré varios problemas como que no contiene todos los temas de cada disco (no hay repeticiones por ejemplo, si el track X está en uno en vivo no aparece quizá en su versión de estudio), hay errores de variado tipo, no está separado el nro de track ni duración de cada track, faltaba el disco póstumo de principios de 2020 (lo tuve que agregar 💪), etc. así que esperamos con quizá un poco más de trabajo, obtener más información desde spotify.

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,
                                                      client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
#Esconder el request response (headers, get, etc)
sp.trace=False 

In [5]:
artistas_id = {'Luis Alberto Spinetta': '', 'Invisible': '', 
               'Spinetta Jade': '', 
               'Almendra': '',
               'Pescado Rabioso': '',
               'Spinetta y los socios del desierto': ''}

In [6]:
#bandas_dict = {'Luis Alberto Spinetta': {'uri': ''},  
#               'Invisible': {'uri': ''}, 
#               'Spinetta Jade': {'uri': ''}, 
#              'Almendra': {'uri': ''}, 
#               'Pescado Rabioso': {'uri': ''}, 
#               'Spinetta y los socios del desierto': {'uri': ''}}

Veamos un sample de la respuesta

In [7]:
sp.search(q='Almendra', type='artist')

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Almendra&type=artist&offset=0&limit=10',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7x2a9uyqlWbE9LwcoQWDTo'},
    'followers': {'href': None, 'total': 155185},
    'genres': ['argentine rock',
     'latin alternative',
     'latin rock',
     'rock en espanol',
     'rock nacional'],
    'href': 'https://api.spotify.com/v1/artists/7x2a9uyqlWbE9LwcoQWDTo',
    'id': '7x2a9uyqlWbE9LwcoQWDTo',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ca4882e0e7595714deeaefe3f66b9c17680c7d3c',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/31fd6d92426b3f223eb0a3dcd19468d94a002eac',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/205be44e74d4e045acb6e4a5dd495adab7caf5d4',
      'width': 64}],
    'name': 'Almendra',
    'popularity': 54,
    'type': 'artist',
    'uri': 'spotify:artist:7x2a9uyqlWbE9LwcoQWDTo'},
   {'external

From Spotipy docs: Spotipy supports a number of different ID types:

        Spotify URI - The resource identifier that you can enter, for example, in the Spotify Desktop client’s search box to locate an artist, album, or track. Example: spotify:track:6rqhFgbbKwnb9MLmUQDhG6
        Spotify URL - An HTML link that opens a track, album, app, playlist or other Spotify resource in a Spotify client. Example: http://open.spotify.com/track/6rqhFgbbKwnb9MLmUQDhG6
        Spotify ID - A base-62 number that you can find at the end of the Spotify URI (see above) for an artist, track, album, etc. Example: 6rqhFgbbKwnb9MLmUQDhG6

In general, any Spotipy method that needs an artist, album, track or playlist ID will accept ids in any of the above form

***

Spotipy nos devuelve un diccionario medio nesteado con información variada de los artistas que matchean. Dada la popularidad del flaco, vamos a limitarnos al primer resultado, si estamos pifiando, luego lo sabremos.
Vamos a obtener el **uri** asignado por spotify a cada artista para luego buscar todos respectivos álbumes y tracks.

In [8]:
for banda in artistas_id:
    artistas_id[banda] = sp.search(q=banda, type='artist', limit=1)[
        'artists']['items'][0]['uri']

artistas_id

{'Luis Alberto Spinetta': 'spotify:artist:1MuQ2m2tg7naeRGAOxYZer',
 'Invisible': 'spotify:artist:3FjdJbt6Myq32uv7P4owM1',
 'Spinetta Jade': 'spotify:artist:3WxVICwFDAWMTWH8sELmRe',
 'Almendra': 'spotify:artist:7x2a9uyqlWbE9LwcoQWDTo',
 'Pescado Rabioso': 'spotify:artist:3q1NXsv9XypOUCJfEatXH9',
 'Spinetta y los socios del desierto': 'spotify:artist:7F5m9Jw4sg853wTDmmJvLi'}

Obtengamos los discos: artist_albums devuelve un dict donde key='items' es una lista con cada disco. De cada disco nos interesa  `uri`, `name`, `album_type`, `total_tracks` y `release_date`. La API tiene un límite de 50 resultados por request. Por otro lado spotify tiene mal catalogado varios discos (figuran como albums pero son compilados), etc.

In [106]:
def get_albums_info(artist_id, album_type=None):
    '''
    artist_id: spotify id (https://spotipy.readthedocs.io/en/2.7.1/#ids-uris-and-urls)
    album_type: 'album', 'single', 'appears_on', 'compilation'. 
    Note that if album_type > 1 element, it must be in a comma separated string as in 'album,single'
    '''
    albums = []
    next_p = '_'
    offset = 0
    while next_p:
        alb_page = sp.artist_albums(
            artist_id, limit=50, offset=offset, album_type=album_type)
        albums.extend([(album['artists'][0]['name'], album['name'], album['album_type'],
                        album['release_date'], album['total_tracks'], album['uri']) for album in alb_page['items']])
        offset += 50
        next_p = alb_page['next']
    return albums

In [163]:
full_list = []
for album_url in artistas_id.values():
    full_list.extend(get_albums_info(artist_id=album_url, album_type='album,single'))

In [164]:
albums_df = pd.DataFrame(data=full_list, columns=[
                         'Artista', 'Disco', 'Tipo', 'Año', 'Total tracks', 'uri'])
albums_df['Año'] = albums_df['Año'].apply(lambda x: x[:4])
albums_df

,Artista,Disco,Tipo,Año,Total tracks,uri
0,Luis Alberto Spinetta,Ya No Mires Atrás,album,2020,7,spotify:album:6o3S8QBSO9oVNAbcVxdJsa
1,Luis Alberto Spinetta,Spinetta y Las Bandas Eternas,album,2010,48,spotify:album:3kuu5DGs4n815JjDI5dlNx
2,Luis Alberto Spinetta,Sí o Sí - Diario del Rock Argentino - Spinetta,album,2008,18,spotify:album:4F6R2CwxpEbdCkZuP6j65E
3,Luis Alberto Spinetta,Un Mañana,album,2008,12,spotify:album:4x9cchTAtN2KEyoE3Mg7FT
4,Fito Paez,La La La,album,2007,20,spotify:album:7MF4MER3ediE5kYLD6NrQ2
5,Luis Alberto Spinetta,Obras Cumbres,album,2006,35,spotify:album:2jRV6FSPqiJaNSwgVBD4G4
6,Luis Alberto Spinetta,Pan,album,2005,12,spotify:album:0xF4fvy8iMh3sT2O4bSgjA
7,Luis Alberto Spinetta,Para Los Arboles,album,2003,12,spotify:album:2Sx9o6kk0gRleobOzhSxkd
8,Luis Alberto Spinetta,Argentina Sorgo Films (Live At Obras / 2001),album,2002,12,spotify:album:40YUyQFHCZ4JaRbfZlMras
9,Luis Alberto Spinetta,Silver Sorgo,album,2001,12,spotify:album:0iSxQ0pzlYkXTOJdJ6rXWa


No es perfecto, pero es lo que hay. Vamos a sacar compilados y algunos repetidos.

In [165]:
albums_df.drop([2,4, 5, 10, 12, 17, 33, 39, 40, 41, 42, 43, 47, 49], inplace=True)

Vamos también a dropear los discos en vivo y Singles/EPs por el momento y guardar en otro df

In [166]:
albums_studio_df = albums_df.drop(albums_df[albums_df['Tipo'] == 'single'].index)
albums_studio_df.drop([1,8,11,15, 25, 45], inplace=True)

In [167]:
albums_studio_df

,Artista,Disco,Tipo,Año,Total tracks,uri
0,Luis Alberto Spinetta,Ya No Mires Atrás,album,2020,7,spotify:album:6o3S8QBSO9oVNAbcVxdJsa
3,Luis Alberto Spinetta,Un Mañana,album,2008,12,spotify:album:4x9cchTAtN2KEyoE3Mg7FT
6,Luis Alberto Spinetta,Pan,album,2005,12,spotify:album:0xF4fvy8iMh3sT2O4bSgjA
7,Luis Alberto Spinetta,Para Los Arboles,album,2003,12,spotify:album:2Sx9o6kk0gRleobOzhSxkd
9,Luis Alberto Spinetta,Silver Sorgo,album,2001,12,spotify:album:0iSxQ0pzlYkXTOJdJ6rXWa
13,Luis Alberto Spinetta,Fuego Gris,album,1993,17,spotify:album:5I7prWCiB53GD5hgkI0Br3
14,Luis Alberto Spinetta,Peluson Of Milk,album,1991,15,spotify:album:0sEqp7Del2dp8HmXE8Geqv
16,Luis Alberto Spinetta,Don Lucero,album,1989,9,spotify:album:4L5xTcKgy8CrAc2H1kN9VY
18,Luis Alberto Spinetta,Tester de Violencia,album,1988,11,spotify:album:3TU8Q7HQS3lNjkVIz5J2yv
19,Fito Paez,La La La,album,1986,20,spotify:album:7e9eBk99v3kYbGO79ZKKVz


In [171]:
# spotify_raw_df = pd.DataFrame()
# for an_album_title, an_album_id in albums_ids.items():
#     a_dataframe = pd.DataFrame(sp.album_tracks(an_album_id)['items'])
#     a_dataframe['album'] = an_album_title
#     if not spotify_raw_df.shape[0]:
#         spotify_raw_df = pd.DataFrame(a_dataframe)
#     else:
#         spotify_raw_df = pd.concat([spotify_raw_df, a_dataframe])

In [ ]:
# for row in albums_studio_df.itertuples(index=False):
#     a_dataframe = pd.DataFrame(sp.album_tracks(an_album_id)['items'])
#     a_dataframe['album'] = an_album_title
#     if not spotify_raw_df.shape[0]:
#         spotify_raw_df = pd.DataFrame(a_dataframe)
#     else:
#         spotify_raw_df = pd.concat([spotify_raw_df, a_dataframe])

In [ ]:
a = sp.album_tracks(albums_studio_df['uri'].iloc[4])['items']
a